In [111]:
# Install requiered packages
!pip install -U bugswarm-common

You should consider upgrading via the '/home/aouinti/semester-project-commit2vec/venv/bin/python3 -m pip install --upgrade pip' command.


In [97]:
from collections import defaultdict
from bugswarm.common.rest_api.database_api import DatabaseAPI
import requests
from tqdm import tqdm
from unidiff import PatchSet
from typing import List
import json

session = requests.Session()
bugswarmapi = DatabaseAPI(token="2vFV-ZCG70az8Fg84uNBvXw0ICnthMRvV83APgAjICY")

In [112]:
def extract_added_removed_code(hunk):
    """
    Extract added and removed lines
    """
    added_lines = []
    removed_lines = []
    for line in hunk:
        if line.is_added:
            added_lines.append(line.value.strip())
        if line.is_removed:
            removed_lines.append(line.value.strip())
    return added_lines, removed_lines


def get_diff(repo: str, commit_sha: str):
    """
    Get a dict where keys are filename and value a dict with added_code and removed_code
    added_code and removed_code are list of strings
    """
    files = {}
    req = session.get(f"https://github.com/{repo}/commit/{commit_sha}.diff")
    patched_files = PatchSet(req.text)
    for patched_file in patched_files:
        filepath = patched_file.source_file[2:]  # remove a/
        for hunk in patched_file:
            try : 
                added_code, removed_code = extract_added_removed_code(hunk)
                files[filepath] = {
                    "added_code": added_code,
                    "removed_code": removed_code
                }
            except : 
                continue 
    return files

print(json.dumps(get_diff("gwtbootstrap3/gwtbootstrap3", "c07f968e099d963eed195c7608487c8515393657"), indent=4))

{
    ".travis.yml": {
        "added_code": [
            "- openjdk8"
        ],
        "removed_code": [
            "- oraclejdk8"
        ]
    },
    "README.md": {
        "added_code": [
            "### Final Release",
            "* 1.0.0 - Released on 26 August 2019.",
            "* Based on Bootstrap v3.4.0",
            "* [Demo](http://gwtbootstrap3.github.io/gwtbootstrap3-demo/) - The GWTBootstrap3 1.0.0 Demo.",
            "* [API Docs](http://gwtbootstrap3.github.io/gwtbootstrap3-demo/apidocs) - The GWTBootstrap3 1.0.0 API Javadoc."
        ],
        "removed_code": [
            "### Current Release",
            "* 0.9.4 - Released on 21 February 2017.",
            "* Based on Bootstrap v3.3.7",
            "* [Demo](http://gwtbootstrap3.github.io/gwtbootstrap3-demo/) - The GWTBootstrap3 0.9.4 Demo.",
            "* [API Docs](http://gwtbootstrap3.github.io/gwtbootstrap3-demo/apidocs) - The GWTBootstrap3 0.9.4 API Javadoc.",
            "### Current Snapshot",
  

In [113]:
ids, labels, msgs, codes = [],[],[],[]
f = open('bugswarm_json')
# return the json data into a dictionary 
data = json.load(f)
for i, artifact in tqdm(enumerate(data)):   
    for job, label in [("failed_job", 0), ("passed_job", 1)]:
        ids.append(artifact[job]["trigger_sha"])
        labels.append(label)
        msgs.append(f"Commit msg for {artifact[job]['trigger_sha']}")
        codes.append(
            [
                diff
                for _, diff in get_diff(artifact["repo"], artifact[job]["trigger_sha"]).items()
            ]
        )

625it [01:49,  5.68it/s]


In [134]:
import re
import pickle
for code in codes:
    for file in code:
        for key, val in file.items():
            if not val:
                continue
            for i in range(len(val)):
                val[i] = re.sub(" +", " ", val[i])
                val[i] = re.sub(r'\t', "", val[i])
data = ids,labels,msgs,codes
bugswarm_training_dataset = open("oussama_output", "wb")
pickle.dump(data,bugswarm_training_dataset)
bugswarm_training_dataset.close() 

In [196]:
# visualization & debugging cell 
# visualize the training pickle file we just created  
with open('oussama_output', 'rb') as f:
    prepared_dataset = pickle.load(f) 
#print("length of our dataset:", len(prepared_dataset))

# visualize the training pickle file of CC2Vec
with open('data+model/data/jit/qt_train.pkl','rb') as f1 :
    cc2vec_dataset = pickle.load(f1) 
#print("length of CC2Vec dataset:",len(cc2vec_dataset))

#print("Compare the 2 datasets: ", "\n our dataset: \n",prepared_dataset[3][0])
#print(  "\n cc2vec dataset:" ,cc2vec_dataset[3][0])

#visualize the dictionary used by CC2Vec 
with open('data+model/data/jit/qt_dict.pkl',  'rb') as f2:
    cc2vec_dict = pickle.load(f2)
print ("length of cc2vec dictionary", len(cc2vec_dict)) 
#visualizing CC2Vec dictionary elements
print (type(cc2vec_dict[1]))
print(list(cc2vec_dict[1])[:2])
nb_printed_elements=0
for k,v in cc2vec_dict[1].items() : 
    if nb_printed_elements<10 : 
        print(k,v)
        nb_printed_elements +=1 

length of cc2vec dictionary 2
<class 'dict'>
['\x1athan', '!']
than 0
! 1
# 2
$ 3
% 4
& 5
' 6
( 7
) 8
* 9


# Use CC2Vec

In [153]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='2, 3'
import gc
gc.collect()
torch.cuda.empty_cache()
!python CC2Vec/jit_cc2ftr.py -train -train_data oussama_output -test_data oussama_output -dictionary_data data+model/data/jit/qt_dict.pkl

100%|███████████████████████████████████████████| 10/10 [00:11<00:00,  1.18s/it]
Training: Epoch 1 / 50 -- Total loss: 9.743361
100%|███████████████████████████████████████████| 10/10 [00:10<00:00,  1.10s/it]
Training: Epoch 2 / 50 -- Total loss: 9.741258
100%|███████████████████████████████████████████| 10/10 [00:11<00:00,  1.13s/it]
Training: Epoch 3 / 50 -- Total loss: 9.739617
100%|███████████████████████████████████████████| 10/10 [00:10<00:00,  1.08s/it]
Training: Epoch 4 / 50 -- Total loss: 9.738098
100%|███████████████████████████████████████████| 10/10 [00:12<00:00,  1.20s/it]
Training: Epoch 5 / 50 -- Total loss: 9.736253
100%|███████████████████████████████████████████| 10/10 [00:12<00:00,  1.30s/it]
Training: Epoch 6 / 50 -- Total loss: 9.734410
100%|███████████████████████████████████████████| 10/10 [00:12<00:00,  1.22s/it]
Training: Epoch 7 / 50 -- Total loss: 9.732187
100%|███████████████████████████████████████████| 10/10 [00:12<00:00,  1.21s/it]
Training: Epoch 8 / 50 

In [156]:
!python CC2Vec/jit_cc2ftr.py -predict -predict_data oussama_output -dictionary_data data+model/data/jit/qt_dict.pkl -load_model data+model/model/jit/qt_cc2ftr.pt -name oussama_output_2.pkl

100%|███████████████████████████████████████| 1250/1250 [04:24<00:00,  4.73it/s]
--------------------------------------------------------------------------------
--------------------------Finish the extracting process-------------------------
--------------------------------------------------------------------------------


In [182]:
with open('oussama_output_2.pkl', 'rb') as f:
    extracted_vectors= pickle.load(f)
print(extracted_vectors.shape)

#showing that CC2Vec model isn't working properly 
similar_vectors=0 
for i in range (0, extracted_vectors.shape[0]-1):
    if extracted_vectors[i].all() ==extracted_vectors[i+1].all() : 
        similar_vectors +=1 
print(similar_vectors+1)
print(extracted_vectors[0])

(1250, 196)
1250
[0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.0000000e+02
 1.0000000e+02 1.0000000e+02 1.0000000e+02 1.0000000e+02 1.0